# Preprocessing

## adding necessary tools

first we install dgl

In [1]:
#installing deep graph library
!pip3 install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.5 MB/s eta 0:00:00


now installing dglgo for some more functions

In [2]:
!pip install dglgo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 34.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 40.1 MB/s eta 0:00:00
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl

In this part we import and add some tools and modules

In [3]:
%matplotlib inline
import os #to create and dealing with directory

os.environ["DGLBACKEND"] = "pytorch"
import dgl #deep graph library
import numpy as np #numerical functions
import networkx as nx #to represent graph and dealing with graph
import torch #for neural network
import torch.nn as nn
import dgl.function as fn #to catch functions
import torch.nn.functional as F #neural network functions
import shutil #help us import dataset
from torch.utils.data import DataLoader #to load data
import cloudpickle
from dgl.nn import GraphConv
from sklearn import preprocessing
import math #for some mathematical functions

## loading dataset

using ESOL dataset.<br>
ESOL is a small dataset consisting of water solubility data for some compounds.

By using shutil we can bring dataset from origin location

In [4]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir + "graph_regression.zip"
shutil.unpack_archive(path_save, folder_data_temp)

Defining a custom pytorch regression dataset class

In [5]:
""" Regression Dataset """
class DGLDatasetReg(torch.utils.data.Dataset):
    def __init__(self, address, transform=None, train=False, scaler=None, scaler_regression=None):
            self.train = train
            self.scaler = scaler
            self.data_set, train_labels_masks_globals = dgl.load_graphs(address+".bin")
            num_graphs = len(self.data_set)
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1) #labels of graph
            self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1) #mask of graph
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1) #globals of graph
            self.transform = transform
            self.scaler_regression = scaler_regression
    def scaler_method(self):
        if self.train:
            scaler = preprocessing.StandardScaler().fit(self.labels) #in order to scale data
            self.scaler = scaler
        return self.scaler
    def __len__(self):
        return len(self.data_set)
    def __getitem__(self, idx):
        if self.scaler_regression:
            """ With Scaler"""
            return  self.data_set[idx], torch.tensor(self.scaler.transform(self.labels)[idx]).float(), self.masks[idx], self.globals[idx]
        else:
            """ Without Scaler """
            return  self.data_set[idx], self.labels[idx].float(), self.masks[idx], self.globals[idx]

performing train and test and split according to file<br>
we bring data from its origin location. at origin location there were 3 scaffold which represent on scaffold in dataset so we choose first one of scaffold0 and then by DGLDataset we bring train and test and validation.<br>
at this step we need to scale train set using Standard Scaler from SKlearn.preprocessing which we defined earlier at defining Regression dataset class.<br>
then according to scaler which we scaled train set, we scale validation set and test set

In [6]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0) #str(0) is because we use scaffold_0 from main datasets at source

train_set = DGLDatasetReg(address=path_data_temp+"_train")
scaler= train_set.scaler_method() #scaling train set
val_set = DGLDatasetReg(address=path_data_temp+"_val", scaler = scaler) #scaling validation according to scaled train set
test_set = DGLDatasetReg(address=path_data_temp+"_test", scaler = scaler) #scaling test according to scaled train set

print(len(train_set), len(val_set), len(test_set))

902 112 114


At this part we catch graphs, labels, masks, globals from batches tuple.


In [7]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals



then we define a loader with batch size of 32 since dataset is small.<br>
we'll define 3 dataloaders for train and validation and test, batch size for each is 32 and collate_fn is according to collate we defined earlier before defining loader

In [8]:
def loader(batch_size = 32):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [9]:
#defining new train test validation by dataloader function
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=128)

# Model Defining

Now it's time to define model,<br>
number of tasks in this dataset is 1 and we set number of epochs to 100 which looks enough for this kind of task. global size is 200 which represents global feature of each graph and patience is 10 which is Number of steps to wait if the model performance on the validation set does not improve.<br>
for configuraton it's better to set node_feature_size to 127, edge_feature_size to 12 and hidden size to 100 in order to avoid further problems.

In [10]:
#ESOL has only 1 task
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}

## defining graph sage

In [11]:
#defining Graph sage class from scratch
class SAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat): #in_feat is input feature size in integer and out_feat is output feature size in integer
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h): # g is input graph and h is input node feature
        with g.local_scope(): #local_scope is to stop out-place mutation to the feature data not reflecting original graph
            g.ndata["h"] = h
            # message passing step
            g.update_all(
                message_func=fn.copy_u("h", "m"), #perorming message function by copy_u function from dgl
                reduce_func=fn.mean("m", "h_N"), #performing reduce function by mean
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1) #concating
            return self.linear(h_total)

for this project we'll use 20 models

## defining models

model1<br>
using graph convolutional<br>
2 layer<br>
no drop out

In [12]:
class model1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True) #allow_zero_in_degree is dou to avoid further errors because of 0 degree nodes(isolated nodes)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]) #first layer
        h = F.relu(h) #relu as activaton function
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model2<br>
using graph sage<br>
2 layer<br>
no drop out

In [13]:
class model2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]) #first layer
        h = F.relu(h) #relu as activaton function
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model3<br>
using graph convolutional<br>
3 layer<br>
no drop out

In [14]:
class model3(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model4<br>
using graph sage<br>
3 layer<br>
no drop out

In [15]:
class model4(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.num_tasks)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model5<br>
using graph convolutional<br>
4 layer<br>
no drop out

In [16]:
class model5(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model6<br>
using graph convolutional<br>
4 layer<br>
dropout = 0.1

In [17]:
class model6(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.1)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model7<br>
using graph convolutional<br>
4 layer<br>
dropout = 0.2

In [18]:
class model7(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.2)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model8<br>
using graph convolutional<br>
4 layer<br>
dropout = 0.25

In [19]:
class model8(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.25)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model9<br>
using graph sage<br>
4 layer<br>
dropout = 0

In [20]:
class model9(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model10<br>
using graph sage<br>
4 layer<br>
dropout = 0.1

In [21]:
class model10(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.1)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model11<br>
using graph sage<br>
4 layer<br>
dropout = 0.2

In [22]:
class model11(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.2)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model12<br>
using graph sage<br>
4 layer<br>
dropout = 0.25

In [23]:
class model12(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.25)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model13<br>
using graph convolutional<br>
5 layer<br>
dropout = 0

In [24]:
class model13(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv5 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model14<br>
using graph convolutional<br>
5 layer<br>
dropout = 0.1

In [25]:
class model14(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv5 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.1)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model15<br>
using graph convolutional<br>
5 layer<br>
dropout = 0.2

In [26]:
class model15(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv5 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.2)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model16<br>
using graph convolutional<br>
5 layer<br>
dropout = 0.25

In [27]:
class model16(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv5 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.25)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model17<br>
using graph sage<br>
5 layer<br>
dropout = 0

In [86]:
class model17(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv5 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model18<br>
using graph sage<br>
5 layer<br>
dropout = 0.1

In [87]:
class model18(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv5 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.1)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model19<br>
using graph sage<br>
5 layer<br>
dropout = 0.2

In [88]:
class model19(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv5 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.2)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

model20<br>
using graph sage<br>
5 layer<br>
dropout = 0.25

In [89]:
class model20(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv4 = SAGEConv(self.hidden_size, self.hidden_size)
        self.conv5 = SAGEConv(self.hidden_size, self.num_tasks)

        self.dropout = nn.Dropout(p=0.25)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

# Above

Function to Compute Score of the Model

In [32]:
def compute_score(model, data_loader, scaler, val_size, num_tasks):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
      prediction = model(mol_dgl_graph, globals)
      loss = loss_sum(prediction, labels)
      final_loss += loss.item()
    final_loss /= val_size
    final_loss = math.sqrt(final_loss)
  return final_loss / num_tasks

loss function

In [33]:
def loss_func(output, label, mask, num_tasks):
  pos_weight = torch.ones((1, num_tasks))
  pos_weight
  criterion = nn.MSELoss(reduction='none')
  loss = mask*criterion(output,label)
  loss = loss.sum() / mask.sum()
  return loss

training function

In [34]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [35]:
def train_evaluate():

    model = model1(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

Function to compute test set score of the final saved model

In [36]:
def test_evaluate():
    final_model = model1(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

**Now it's time to evaluate models**

model1

In [37]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 12.622 | Valid Score: 4.406
 
Epoch: 1/100 | Best Valid Score Until Now: 4.406 

Save checkpoint
Epoch: 2/100 | Training Loss: 11.095 | Valid Score: 4.361
 
Epoch: 2/100 | Best Valid Score Until Now: 4.361 

Save checkpoint
Epoch: 3/100 | Training Loss: 12.856 | Valid Score: 4.318
 
Epoch: 3/100 | Best Valid Score Until Now: 4.318 

Save checkpoint
Epoch: 4/100 | Training Loss: 10.727 | Valid Score: 4.275
 
Epoch: 4/100 | Best Valid Score Until Now: 4.275 

Save checkpoint
Epoch: 5/100 | Training Loss: 11.567 | Valid Score: 4.233
 
Epoch: 5/100 | Best Valid Score Until Now: 4.233 

Save checkpoint
Epoch: 6/100 | Training Loss: 10.209 | Valid Score: 4.192
 
Epoch: 6/100 | Best Valid Score Until Now: 4.192 

Save checkpoint
Epoch: 7/100 | Training Loss: 12.346 | Valid Score: 4.152
 
Epoch: 7/100 | Best Valid Score Until Now: 4.152 

Save checkpoint
Epoch: 8/100 | Training Loss: 9.490 | Valid Score: 4.111
 
Epoch: 8/100 | Best Valid Score Unti

model2

In [38]:
def train_evaluate():

    model = model2(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [39]:
def test_evaluate():
    final_model = model2(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [40]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 11.422 | Valid Score: 4.275
 
Epoch: 1/100 | Best Valid Score Until Now: 4.275 

Save checkpoint
Epoch: 2/100 | Training Loss: 10.491 | Valid Score: 4.238
 
Epoch: 2/100 | Best Valid Score Until Now: 4.238 

Save checkpoint
Epoch: 3/100 | Training Loss: 10.682 | Valid Score: 4.200
 
Epoch: 3/100 | Best Valid Score Until Now: 4.200 

Save checkpoint
Epoch: 4/100 | Training Loss: 10.185 | Valid Score: 4.161
 
Epoch: 4/100 | Best Valid Score Until Now: 4.161 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.861 | Valid Score: 4.120
 
Epoch: 5/100 | Best Valid Score Until Now: 4.120 

Save checkpoint
Epoch: 6/100 | Training Loss: 10.780 | Valid Score: 4.078
 
Epoch: 6/100 | Best Valid Score Until Now: 4.078 

Save checkpoint
Epoch: 7/100 | Training Loss: 9.900 | Valid Score: 4.032
 
Epoch: 7/100 | Best Valid Score Until Now: 4.032 

Save checkpoint
Epoch: 8/100 | Training Loss: 9.022 | Valid Score: 3.983
 
Epoch: 8/100 | Best Valid Score Until 

model3

In [41]:
def train_evaluate():

    model = model3(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [42]:
def test_evaluate():
    final_model = model3(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [43]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 9.421 | Valid Score: 3.861
 
Epoch: 1/100 | Best Valid Score Until Now: 3.861 

Save checkpoint
Epoch: 2/100 | Training Loss: 8.668 | Valid Score: 3.801
 
Epoch: 2/100 | Best Valid Score Until Now: 3.801 

Save checkpoint
Epoch: 3/100 | Training Loss: 8.403 | Valid Score: 3.741
 
Epoch: 3/100 | Best Valid Score Until Now: 3.741 

Save checkpoint
Epoch: 4/100 | Training Loss: 7.413 | Valid Score: 3.682
 
Epoch: 4/100 | Best Valid Score Until Now: 3.682 

Save checkpoint
Epoch: 5/100 | Training Loss: 8.017 | Valid Score: 3.622
 
Epoch: 5/100 | Best Valid Score Until Now: 3.622 

Save checkpoint
Epoch: 6/100 | Training Loss: 7.593 | Valid Score: 3.561
 
Epoch: 6/100 | Best Valid Score Until Now: 3.561 

Save checkpoint
Epoch: 7/100 | Training Loss: 7.050 | Valid Score: 3.501
 
Epoch: 7/100 | Best Valid Score Until Now: 3.501 

Save checkpoint
Epoch: 8/100 | Training Loss: 6.572 | Valid Score: 3.441
 
Epoch: 8/100 | Best Valid Score Until Now: 

model4

In [44]:
def train_evaluate():

    model = model4(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [45]:
def test_evaluate():
    final_model = model4(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [46]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 12.114 | Valid Score: 4.187
 
Epoch: 1/100 | Best Valid Score Until Now: 4.187 

Save checkpoint
Epoch: 2/100 | Training Loss: 10.020 | Valid Score: 4.151
 
Epoch: 2/100 | Best Valid Score Until Now: 4.151 

Save checkpoint
Epoch: 3/100 | Training Loss: 9.886 | Valid Score: 4.111
 
Epoch: 3/100 | Best Valid Score Until Now: 4.111 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.341 | Valid Score: 4.065
 
Epoch: 4/100 | Best Valid Score Until Now: 4.065 

Save checkpoint
Epoch: 5/100 | Training Loss: 12.024 | Valid Score: 4.009
 
Epoch: 5/100 | Best Valid Score Until Now: 4.009 

Save checkpoint
Epoch: 6/100 | Training Loss: 9.125 | Valid Score: 3.938
 
Epoch: 6/100 | Best Valid Score Until Now: 3.938 

Save checkpoint
Epoch: 7/100 | Training Loss: 9.049 | Valid Score: 3.852
 
Epoch: 7/100 | Best Valid Score Until Now: 3.852 

Save checkpoint
Epoch: 8/100 | Training Loss: 9.049 | Valid Score: 3.750
 
Epoch: 8/100 | Best Valid Score Until No

model5

In [47]:
def train_evaluate():

    model = model5(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [48]:
def test_evaluate():
    final_model = model5(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [49]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 11.388 | Valid Score: 4.111
 
Epoch: 1/100 | Best Valid Score Until Now: 4.111 

Save checkpoint
Epoch: 2/100 | Training Loss: 9.754 | Valid Score: 4.016
 
Epoch: 2/100 | Best Valid Score Until Now: 4.016 

Save checkpoint
Epoch: 3/100 | Training Loss: 8.925 | Valid Score: 3.924
 
Epoch: 3/100 | Best Valid Score Until Now: 3.924 

Save checkpoint
Epoch: 4/100 | Training Loss: 8.831 | Valid Score: 3.832
 
Epoch: 4/100 | Best Valid Score Until Now: 3.832 

Save checkpoint
Epoch: 5/100 | Training Loss: 8.845 | Valid Score: 3.740
 
Epoch: 5/100 | Best Valid Score Until Now: 3.740 

Save checkpoint
Epoch: 6/100 | Training Loss: 8.897 | Valid Score: 3.646
 
Epoch: 6/100 | Best Valid Score Until Now: 3.646 

Save checkpoint
Epoch: 7/100 | Training Loss: 7.727 | Valid Score: 3.549
 
Epoch: 7/100 | Best Valid Score Until Now: 3.549 

Save checkpoint
Epoch: 8/100 | Training Loss: 6.695 | Valid Score: 3.451
 
Epoch: 8/100 | Best Valid Score Until Now:

model6

In [50]:
def train_evaluate():

    model = model6(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [51]:
def test_evaluate():
    final_model = model6(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [52]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 12.988 | Valid Score: 4.156
 
Epoch: 1/100 | Best Valid Score Until Now: 4.156 

Save checkpoint
Epoch: 2/100 | Training Loss: 9.844 | Valid Score: 4.058
 
Epoch: 2/100 | Best Valid Score Until Now: 4.058 

Save checkpoint
Epoch: 3/100 | Training Loss: 9.624 | Valid Score: 3.961
 
Epoch: 3/100 | Best Valid Score Until Now: 3.961 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.777 | Valid Score: 3.863
 
Epoch: 4/100 | Best Valid Score Until Now: 3.863 

Save checkpoint
Epoch: 5/100 | Training Loss: 8.733 | Valid Score: 3.760
 
Epoch: 5/100 | Best Valid Score Until Now: 3.760 

Save checkpoint
Epoch: 6/100 | Training Loss: 7.743 | Valid Score: 3.654
 
Epoch: 6/100 | Best Valid Score Until Now: 3.654 

Save checkpoint
Epoch: 7/100 | Training Loss: 7.612 | Valid Score: 3.556
 
Epoch: 7/100 | Best Valid Score Until Now: 3.556 

Save checkpoint
Epoch: 8/100 | Training Loss: 7.367 | Valid Score: 3.455
 
Epoch: 8/100 | Best Valid Score Until Now:

model7

In [53]:
def train_evaluate():

    model = model7(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [54]:
def test_evaluate():
    final_model = model7(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [55]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 8.794 | Valid Score: 3.893
 
Epoch: 1/100 | Best Valid Score Until Now: 3.893 

Save checkpoint
Epoch: 2/100 | Training Loss: 8.308 | Valid Score: 3.785
 
Epoch: 2/100 | Best Valid Score Until Now: 3.785 

Save checkpoint
Epoch: 3/100 | Training Loss: 8.978 | Valid Score: 3.676
 
Epoch: 3/100 | Best Valid Score Until Now: 3.676 

Save checkpoint
Epoch: 4/100 | Training Loss: 7.929 | Valid Score: 3.564
 
Epoch: 4/100 | Best Valid Score Until Now: 3.564 

Save checkpoint
Epoch: 5/100 | Training Loss: 7.427 | Valid Score: 3.453
 
Epoch: 5/100 | Best Valid Score Until Now: 3.453 

Save checkpoint
Epoch: 6/100 | Training Loss: 6.735 | Valid Score: 3.349
 
Epoch: 6/100 | Best Valid Score Until Now: 3.349 

Save checkpoint
Epoch: 7/100 | Training Loss: 5.961 | Valid Score: 3.258
 
Epoch: 7/100 | Best Valid Score Until Now: 3.258 

Save checkpoint
Epoch: 8/100 | Training Loss: 5.954 | Valid Score: 3.173
 
Epoch: 8/100 | Best Valid Score Until Now: 

model8

In [56]:
def train_evaluate():

    model = model8(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [57]:
def test_evaluate():
    final_model = model8(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [58]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 10.529 | Valid Score: 4.108
 
Epoch: 1/100 | Best Valid Score Until Now: 4.108 

Save checkpoint
Epoch: 2/100 | Training Loss: 11.123 | Valid Score: 4.028
 
Epoch: 2/100 | Best Valid Score Until Now: 4.028 

Save checkpoint
Epoch: 3/100 | Training Loss: 9.786 | Valid Score: 3.951
 
Epoch: 3/100 | Best Valid Score Until Now: 3.951 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.105 | Valid Score: 3.873
 
Epoch: 4/100 | Best Valid Score Until Now: 3.873 

Save checkpoint
Epoch: 5/100 | Training Loss: 8.484 | Valid Score: 3.792
 
Epoch: 5/100 | Best Valid Score Until Now: 3.792 

Save checkpoint
Epoch: 6/100 | Training Loss: 7.635 | Valid Score: 3.706
 
Epoch: 6/100 | Best Valid Score Until Now: 3.706 

Save checkpoint
Epoch: 7/100 | Training Loss: 7.394 | Valid Score: 3.614
 
Epoch: 7/100 | Best Valid Score Until Now: 3.614 

Save checkpoint
Epoch: 8/100 | Training Loss: 8.551 | Valid Score: 3.515
 
Epoch: 8/100 | Best Valid Score Until Now

model9

In [59]:
def train_evaluate():

    model = model9(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [60]:
def test_evaluate():
    final_model = model9(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [61]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 10.387 | Valid Score: 4.200
 
Epoch: 1/100 | Best Valid Score Until Now: 4.200 

Save checkpoint
Epoch: 2/100 | Training Loss: 11.241 | Valid Score: 4.173
 
Epoch: 2/100 | Best Valid Score Until Now: 4.173 

Save checkpoint
Epoch: 3/100 | Training Loss: 11.740 | Valid Score: 4.141
 
Epoch: 3/100 | Best Valid Score Until Now: 4.141 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.901 | Valid Score: 4.099
 
Epoch: 4/100 | Best Valid Score Until Now: 4.099 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.868 | Valid Score: 4.041
 
Epoch: 5/100 | Best Valid Score Until Now: 4.041 

Save checkpoint
Epoch: 6/100 | Training Loss: 9.040 | Valid Score: 3.957
 
Epoch: 6/100 | Best Valid Score Until Now: 3.957 

Save checkpoint
Epoch: 7/100 | Training Loss: 9.024 | Valid Score: 3.840
 
Epoch: 7/100 | Best Valid Score Until Now: 3.840 

Save checkpoint
Epoch: 8/100 | Training Loss: 8.116 | Valid Score: 3.679
 
Epoch: 8/100 | Best Valid Score Until No

model10

In [62]:
def train_evaluate():

    model = model10(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [63]:
def test_evaluate():
    final_model = model10(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [64]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 10.399 | Valid Score: 4.169
 
Epoch: 1/100 | Best Valid Score Until Now: 4.169 

Save checkpoint
Epoch: 2/100 | Training Loss: 11.881 | Valid Score: 4.132
 
Epoch: 2/100 | Best Valid Score Until Now: 4.132 

Save checkpoint
Epoch: 3/100 | Training Loss: 11.090 | Valid Score: 4.086
 
Epoch: 3/100 | Best Valid Score Until Now: 4.086 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.754 | Valid Score: 4.023
 
Epoch: 4/100 | Best Valid Score Until Now: 4.023 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.571 | Valid Score: 3.934
 
Epoch: 5/100 | Best Valid Score Until Now: 3.934 

Save checkpoint
Epoch: 6/100 | Training Loss: 8.211 | Valid Score: 3.805
 
Epoch: 6/100 | Best Valid Score Until Now: 3.805 

Save checkpoint
Epoch: 7/100 | Training Loss: 8.119 | Valid Score: 3.629
 
Epoch: 7/100 | Best Valid Score Until Now: 3.629 

Save checkpoint
Epoch: 8/100 | Training Loss: 7.102 | Valid Score: 3.403
 
Epoch: 8/100 | Best Valid Score Until No

model11

In [65]:
def train_evaluate():

    model = model11(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [66]:
def test_evaluate():
    final_model = model11(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [67]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 10.919 | Valid Score: 4.229
 
Epoch: 1/100 | Best Valid Score Until Now: 4.229 

Save checkpoint
Epoch: 2/100 | Training Loss: 10.828 | Valid Score: 4.203
 
Epoch: 2/100 | Best Valid Score Until Now: 4.203 

Save checkpoint
Epoch: 3/100 | Training Loss: 10.538 | Valid Score: 4.173
 
Epoch: 3/100 | Best Valid Score Until Now: 4.173 

Save checkpoint
Epoch: 4/100 | Training Loss: 10.555 | Valid Score: 4.134
 
Epoch: 4/100 | Best Valid Score Until Now: 4.134 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.782 | Valid Score: 4.081
 
Epoch: 5/100 | Best Valid Score Until Now: 4.081 

Save checkpoint
Epoch: 6/100 | Training Loss: 9.736 | Valid Score: 4.007
 
Epoch: 6/100 | Best Valid Score Until Now: 4.007 

Save checkpoint
Epoch: 7/100 | Training Loss: 9.610 | Valid Score: 3.903
 
Epoch: 7/100 | Best Valid Score Until Now: 3.903 

Save checkpoint
Epoch: 8/100 | Training Loss: 8.497 | Valid Score: 3.754
 
Epoch: 8/100 | Best Valid Score Until N

model12

In [68]:
def train_evaluate():

    model = model12(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [69]:
def test_evaluate():
    final_model = model12(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [70]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 11.134 | Valid Score: 4.274
 
Epoch: 1/100 | Best Valid Score Until Now: 4.274 

Save checkpoint
Epoch: 2/100 | Training Loss: 10.980 | Valid Score: 4.245
 
Epoch: 2/100 | Best Valid Score Until Now: 4.245 

Save checkpoint
Epoch: 3/100 | Training Loss: 10.458 | Valid Score: 4.208
 
Epoch: 3/100 | Best Valid Score Until Now: 4.208 

Save checkpoint
Epoch: 4/100 | Training Loss: 10.536 | Valid Score: 4.155
 
Epoch: 4/100 | Best Valid Score Until Now: 4.155 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.695 | Valid Score: 4.082
 
Epoch: 5/100 | Best Valid Score Until Now: 4.082 

Save checkpoint
Epoch: 6/100 | Training Loss: 9.631 | Valid Score: 3.976
 
Epoch: 6/100 | Best Valid Score Until Now: 3.976 

Save checkpoint
Epoch: 7/100 | Training Loss: 8.816 | Valid Score: 3.825
 
Epoch: 7/100 | Best Valid Score Until Now: 3.825 

Save checkpoint
Epoch: 8/100 | Training Loss: 8.574 | Valid Score: 3.623
 
Epoch: 8/100 | Best Valid Score Until N

model13

In [71]:
def train_evaluate():

    model = model13(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [72]:
def test_evaluate():
    final_model = model13(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [73]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 10.528 | Valid Score: 4.139
 
Epoch: 1/100 | Best Valid Score Until Now: 4.139 

Save checkpoint
Epoch: 2/100 | Training Loss: 9.867 | Valid Score: 4.064
 
Epoch: 2/100 | Best Valid Score Until Now: 4.064 

Save checkpoint
Epoch: 3/100 | Training Loss: 10.402 | Valid Score: 3.985
 
Epoch: 3/100 | Best Valid Score Until Now: 3.985 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.242 | Valid Score: 3.899
 
Epoch: 4/100 | Best Valid Score Until Now: 3.899 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.125 | Valid Score: 3.805
 
Epoch: 5/100 | Best Valid Score Until Now: 3.805 

Save checkpoint
Epoch: 6/100 | Training Loss: 8.229 | Valid Score: 3.701
 
Epoch: 6/100 | Best Valid Score Until Now: 3.701 

Save checkpoint
Epoch: 7/100 | Training Loss: 7.333 | Valid Score: 3.585
 
Epoch: 7/100 | Best Valid Score Until Now: 3.585 

Save checkpoint
Epoch: 8/100 | Training Loss: 8.032 | Valid Score: 3.464
 
Epoch: 8/100 | Best Valid Score Until Now

model14

In [74]:
def train_evaluate():

    model = model14(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [75]:
def test_evaluate():
    final_model = model14(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [76]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 11.324 | Valid Score: 4.204
 
Epoch: 1/100 | Best Valid Score Until Now: 4.204 

Save checkpoint
Epoch: 2/100 | Training Loss: 10.852 | Valid Score: 4.140
 
Epoch: 2/100 | Best Valid Score Until Now: 4.140 

Save checkpoint
Epoch: 3/100 | Training Loss: 9.383 | Valid Score: 4.080
 
Epoch: 3/100 | Best Valid Score Until Now: 4.080 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.848 | Valid Score: 4.018
 
Epoch: 4/100 | Best Valid Score Until Now: 4.018 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.221 | Valid Score: 3.946
 
Epoch: 5/100 | Best Valid Score Until Now: 3.946 

Save checkpoint
Epoch: 6/100 | Training Loss: 9.865 | Valid Score: 3.860
 
Epoch: 6/100 | Best Valid Score Until Now: 3.860 

Save checkpoint
Epoch: 7/100 | Training Loss: 7.824 | Valid Score: 3.758
 
Epoch: 7/100 | Best Valid Score Until Now: 3.758 

Save checkpoint
Epoch: 8/100 | Training Loss: 8.832 | Valid Score: 3.646
 
Epoch: 8/100 | Best Valid Score Until Now

model15

In [77]:
def train_evaluate():

    model = model15(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [78]:
def test_evaluate():
    final_model = model15(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [79]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 11.430 | Valid Score: 4.160
 
Epoch: 1/100 | Best Valid Score Until Now: 4.160 

Save checkpoint
Epoch: 2/100 | Training Loss: 10.819 | Valid Score: 4.084
 
Epoch: 2/100 | Best Valid Score Until Now: 4.084 

Save checkpoint
Epoch: 3/100 | Training Loss: 9.609 | Valid Score: 4.007
 
Epoch: 3/100 | Best Valid Score Until Now: 4.007 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.094 | Valid Score: 3.928
 
Epoch: 4/100 | Best Valid Score Until Now: 3.928 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.219 | Valid Score: 3.844
 
Epoch: 5/100 | Best Valid Score Until Now: 3.844 

Save checkpoint
Epoch: 6/100 | Training Loss: 9.022 | Valid Score: 3.746
 
Epoch: 6/100 | Best Valid Score Until Now: 3.746 

Save checkpoint
Epoch: 7/100 | Training Loss: 7.318 | Valid Score: 3.641
 
Epoch: 7/100 | Best Valid Score Until Now: 3.641 

Save checkpoint
Epoch: 8/100 | Training Loss: 6.825 | Valid Score: 3.523
 
Epoch: 8/100 | Best Valid Score Until Now

model16

In [80]:
def train_evaluate():

    model = model16(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [81]:
def test_evaluate():
    final_model = model16(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [82]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 11.795 | Valid Score: 4.215
 
Epoch: 1/100 | Best Valid Score Until Now: 4.215 

Save checkpoint
Epoch: 2/100 | Training Loss: 10.747 | Valid Score: 4.175
 
Epoch: 2/100 | Best Valid Score Until Now: 4.175 

Save checkpoint
Epoch: 3/100 | Training Loss: 10.484 | Valid Score: 4.135
 
Epoch: 3/100 | Best Valid Score Until Now: 4.135 

Save checkpoint
Epoch: 4/100 | Training Loss: 10.208 | Valid Score: 4.093
 
Epoch: 4/100 | Best Valid Score Until Now: 4.093 

Save checkpoint
Epoch: 5/100 | Training Loss: 11.041 | Valid Score: 4.046
 
Epoch: 5/100 | Best Valid Score Until Now: 4.046 

Save checkpoint
Epoch: 6/100 | Training Loss: 10.741 | Valid Score: 3.993
 
Epoch: 6/100 | Best Valid Score Until Now: 3.993 

Save checkpoint
Epoch: 7/100 | Training Loss: 10.296 | Valid Score: 3.926
 
Epoch: 7/100 | Best Valid Score Until Now: 3.926 

Save checkpoint
Epoch: 8/100 | Training Loss: 9.144 | Valid Score: 3.843
 
Epoch: 8/100 | Best Valid Score Unti

model17

In [90]:
def train_evaluate():

    model = model17(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [91]:
def test_evaluate():
    final_model = model17(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [92]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 12.230 | Valid Score: 4.284
 
Epoch: 1/100 | Best Valid Score Until Now: 4.284 

Save checkpoint
Epoch: 2/100 | Training Loss: 12.064 | Valid Score: 4.265
 
Epoch: 2/100 | Best Valid Score Until Now: 4.265 

Save checkpoint
Epoch: 3/100 | Training Loss: 10.234 | Valid Score: 4.246
 
Epoch: 3/100 | Best Valid Score Until Now: 4.246 

Save checkpoint
Epoch: 4/100 | Training Loss: 10.153 | Valid Score: 4.224
 
Epoch: 4/100 | Best Valid Score Until Now: 4.224 

Save checkpoint
Epoch: 5/100 | Training Loss: 12.080 | Valid Score: 4.193
 
Epoch: 5/100 | Best Valid Score Until Now: 4.193 

Save checkpoint
Epoch: 6/100 | Training Loss: 11.239 | Valid Score: 4.143
 
Epoch: 6/100 | Best Valid Score Until Now: 4.143 

Save checkpoint
Epoch: 7/100 | Training Loss: 10.090 | Valid Score: 4.055
 
Epoch: 7/100 | Best Valid Score Until Now: 4.055 

Save checkpoint
Epoch: 8/100 | Training Loss: 9.410 | Valid Score: 3.901
 
Epoch: 8/100 | Best Valid Score Unti

model18

In [93]:
def train_evaluate():

    model = model18(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [94]:
def test_evaluate():
    final_model = model18(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [95]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 10.435 | Valid Score: 4.141
 
Epoch: 1/100 | Best Valid Score Until Now: 4.141 

Save checkpoint
Epoch: 2/100 | Training Loss: 10.658 | Valid Score: 4.107
 
Epoch: 2/100 | Best Valid Score Until Now: 4.107 

Save checkpoint
Epoch: 3/100 | Training Loss: 10.775 | Valid Score: 4.061
 
Epoch: 3/100 | Best Valid Score Until Now: 4.061 

Save checkpoint
Epoch: 4/100 | Training Loss: 9.110 | Valid Score: 3.991
 
Epoch: 4/100 | Best Valid Score Until Now: 3.991 

Save checkpoint
Epoch: 5/100 | Training Loss: 8.727 | Valid Score: 3.879
 
Epoch: 5/100 | Best Valid Score Until Now: 3.879 

Save checkpoint
Epoch: 6/100 | Training Loss: 7.892 | Valid Score: 3.700
 
Epoch: 6/100 | Best Valid Score Until Now: 3.700 

Save checkpoint
Epoch: 7/100 | Training Loss: 7.075 | Valid Score: 3.429
 
Epoch: 7/100 | Best Valid Score Until Now: 3.429 

Save checkpoint
Epoch: 8/100 | Training Loss: 6.055 | Valid Score: 3.102
 
Epoch: 8/100 | Best Valid Score Until No

model19

In [96]:
def train_evaluate():

    model = model19(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [97]:
def test_evaluate():
    final_model = model19(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [98]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 12.036 | Valid Score: 4.285
 
Epoch: 1/100 | Best Valid Score Until Now: 4.285 

Save checkpoint
Epoch: 2/100 | Training Loss: 12.407 | Valid Score: 4.267
 
Epoch: 2/100 | Best Valid Score Until Now: 4.267 

Save checkpoint
Epoch: 3/100 | Training Loss: 10.181 | Valid Score: 4.247
 
Epoch: 3/100 | Best Valid Score Until Now: 4.247 

Save checkpoint
Epoch: 4/100 | Training Loss: 11.299 | Valid Score: 4.221
 
Epoch: 4/100 | Best Valid Score Until Now: 4.221 

Save checkpoint
Epoch: 5/100 | Training Loss: 12.123 | Valid Score: 4.181
 
Epoch: 5/100 | Best Valid Score Until Now: 4.181 

Save checkpoint
Epoch: 6/100 | Training Loss: 11.336 | Valid Score: 4.117
 
Epoch: 6/100 | Best Valid Score Until Now: 4.117 

Save checkpoint
Epoch: 7/100 | Training Loss: 9.540 | Valid Score: 4.013
 
Epoch: 7/100 | Best Valid Score Until Now: 4.013 

Save checkpoint
Epoch: 8/100 | Training Loss: 8.562 | Valid Score: 3.844
 
Epoch: 8/100 | Best Valid Score Until

model20

In [99]:
def train_evaluate():

    model = model20(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")

In [100]:
def test_evaluate():
    final_model = model20(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(val_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [101]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 10.762 | Valid Score: 4.214
 
Epoch: 1/100 | Best Valid Score Until Now: 4.214 

Save checkpoint
Epoch: 2/100 | Training Loss: 11.132 | Valid Score: 4.191
 
Epoch: 2/100 | Best Valid Score Until Now: 4.191 

Save checkpoint
Epoch: 3/100 | Training Loss: 11.228 | Valid Score: 4.160
 
Epoch: 3/100 | Best Valid Score Until Now: 4.160 

Save checkpoint
Epoch: 4/100 | Training Loss: 10.201 | Valid Score: 4.113
 
Epoch: 4/100 | Best Valid Score Until Now: 4.113 

Save checkpoint
Epoch: 5/100 | Training Loss: 9.666 | Valid Score: 4.035
 
Epoch: 5/100 | Best Valid Score Until Now: 4.035 

Save checkpoint
Epoch: 6/100 | Training Loss: 9.019 | Valid Score: 3.901
 
Epoch: 6/100 | Best Valid Score Until Now: 3.901 

Save checkpoint
Epoch: 7/100 | Training Loss: 8.773 | Valid Score: 3.680
 
Epoch: 7/100 | Best Valid Score Until Now: 3.680 

Save checkpoint
Epoch: 8/100 | Training Loss: 6.777 | Valid Score: 3.359
 
Epoch: 8/100 | Best Valid Score Until N